# Loading packages and utilities

In [ ]:
Pkg.add("JLD2")
Pkg.update()

In [ ]:
include("getcorrespondences.jl")

# Load your projective_transform.jl file

In [ ]:
# include your autograder-certified code from homework
include("projective_transform.jl")

Next load the images and convert them into an array for processing. 

In [ ]:
names = ["law"] ## sample file -- tag the logos in the window in first image
im_name = names[1]
# you can swap out the paths for your image!
inpath1 = "$(im_name)1.jpg"
inpath2 = "$(im_name)2.jpg"

## Load images and convert to an Array
im1 = convert(Array, load(inpath1))
im2 = convert(Array, load(inpath2))
nothing

Let us first display Image 1. 

In [ ]:
display(im1)

And now, Image 2.

In [ ]:
display(im2)

In [ ]:
using MIT18065

# includes the functions needed to do the stitching
include("Photostitching.jl")

In [ ]:
## Tag the points in the logos of image one and the window around it
## These correspond to the left most window in image 2 
## Planar surface correspondences work best!
## Get correspondences
# there is only one example file with 8 points, any other number will prompt you to pick points
n = 8 # correspondencs (>= 4)

pointsPath = "$(im_name)_points$(n).jld"

## To pick your own points, comment out the if statement and run just the following:
## (XY1, XY2), plot = getcorrespondences(im1, im2, n)
## save(pointsPath, "XY1", "XY2")
## plot 

if isfile(pointsPath) # if there is a file with points
    # Load saved points
    vars = load(pointsPath)
    println(vars)
    XY1 = vars["XY1"]
    XY2 = vars["XY2"]
else
    # Else open a GUI to let you choose correspondences
    (XY1, XY2), plot = getcorrespondences(im1, im2, n)
    save(pointsPath, "XY1", "XY2")
end

plot

We now compute the Homography Matrix that transforms points in Image 2 to the points in Image 1. This $3 \times 3$ matrix is computed next.

In [ ]:
H21 = projective_transform(XY2, XY1) # 2 --> 1

We also compute the Homography Matrix that transforms points in Image 1 to the points in Image 2. This $3 \times 3$ matrix is computed next.

In [ ]:
H12 = projective_transform(XY1, XY2) # 1 --> 2 

Note that these matrices are not inverses of each other! We will now stitch together Image 1 and Image 2, from the perspective of Image 1 and display the resulting stitched image. This is accomplished using the following code: 

In [ ]:
## Stitch images from perspective 1

I1 = [
    ImageStitcher(im1, eye(3)),
    ImageStitcher(im2, H21)
]

# order = "natural": closet (to chosen persepctive) displayed on top
# order = "reverse": farthest displayed on top
imS1 = stitchImages(I1, order = "reverse") 

Next we  stitch together Image 2 and Image 1, from the perspective of Image 2 and display the resulting image.

In [ ]:
## Stitch images from perspective 2

I2 = [
    ImageStitcher(im2, eye(3)),
    ImageStitcher(im1, H12)
]

# order = "natural": closet (to chosen persepctive) displayed on top
# order = "reverse": farthest displayed on top
imS2 = stitchImages(I2, order = "natural")


We are now done! You can save the images created by executing the cell below. 

In [ ]:
## Save png images of the stitched images
save("$(im_name)_stitched1.png", imS1)
save("$(im_name)_stitched2.png", imS2)